In [3]:
import numpy as np
import pandas as pd
import json

### 게임 결과 데이터 프레임 만들기




1. MatchDto

게임 정보입니다. Info와 metadata로 구성됩니다.

1) metadata

해당 게임 버전과 참가자 아이디를 나타내줍니다.

data_version : 해당 게임 시즌

match_id : match id

participants : 참가자 puuid

2) Info

게임 세부정보입니다. 


game_datetime : 해당 match 시작 시간, Unix timestamp으로 결과 제공

game_length : 해당 match 끝날 때까지의 시간, 단위 : 초

game_version : 해당 match의 게임 version

game_variation : 해당 match에서의 게임 변화

participant : 게임 세부 정보, 참가자마다의 순위 결정시의 정보 제공


2-1) Participant

augment : 증강체

companion : 전략가 정보(기물을 다루는 캐릭터) - content_id(전략가 이름), skin_id(스킨id), species(전략가 캐릭터 이름)으로 구성

gold_left : 게임을 끝마쳤을 때 남은 골드

last_round : 게임을 끝마쳤을 때의 라운드(ex:2-1라운드에 패배시 5)

level : 게임을 끝마쳤을 때의 level

players_eliminated : 자신이 죽인 플레이어 수

puuid : 해당 유저의 puuid

time_eliminated : 해당 유저가 패배할 때까지의 시간

total_damage_to_players : 해당 유저가 다른 유저에게 가한 피해량 총합

traits : 해당 유저 덱의 시너지 효과 정보

units : 해당 유저 덱의 유닛 정보


2-1-1) Traits

name : 시너지 이름

num_units : 덱에 해당 시너지를 가진 유닛 수

style : 시너지 스타일(0=시너지 없음, 1=브론즈 시너지, 2=실버 시너지, 3=골드 시너지, 4=크로마틱 시너지)

tier_current : 활성화된 시너지 티어 

tier_total : 해당 시너지 총 티어



2-1-2) Units

items : 해당 유닛이 가지고 있는 아이템 (list로 제공)

character_id : 유닛 이름

chosen : If a unit is chosen as part of the Fates set mechanic, the chosen trait will be indicated by this field. Otherwise this field is excluded from the response.

name : 유닛 이름, 대부분 빈칸

rarity : 유닛 희귀도, unit cost와 다름

tier : 유닛 tier


In [11]:
file_path='./item data.json'
with open(file_path, "r") as json_file:
    data_dragon = json.load(json_file)
    


In [9]:
class make_df:
    
    def __init__(self, df, version, data_dragon):
        self.df = df
        self.version = version
        self.data_dragon = data_dragon
        self.item_data = pd.DataFrame(self.data_dragon['items'])
        self.champ = pd.DataFrame(pd.DataFrame(self.data_dragon['setData']).iloc[2, 0])
        self.game_id = self.df['metadata']['match_id']
        self.game_result = pd.DataFrame(self.df['info']['participants']).drop(['companion'], axis=1)
        self.game_result['game_id'] = self.game_id
        self.game_result = self.game_result[['game_id', 'puuid', 'placement', 'level', 'gold_left', 'last_round', 'time_eliminated',
                                             'total_damage_to_players', 'augments', 'traits', 'units']]
        self.trait_language = pd.DataFrame({'Korean' : ['민간인', '자동방어체계', '동물특공대', '레이저단', '싸움꾼', '메카:프라임', '위협',
                                                '마스코트', '기상캐스터', '에이스', '결투가', '황소부대', '병기고', '방패대',
                                                '지하세계', '별 수호자', '익살꾼', '선의', '타락', '우세', '무법자', '주문투척자',
                                                '정찰단', '특등사수', '엄호대', '기계유망주', '해커'],
                                   "English" : ['Set8_Civilian', 'Set8_Admin', 'Set8_AnimaSquad', 'Set8_SpaceCorps', 'Set8_Brawler',
                                                 'Set8_ExoPrime', 'Set8_Threat', 'Set8_Mascot', 'Set8_Forecaster', 'Set8_Ace',
                                                 'Set8_Duelist', 'Set8_OxForce', 'Set8_Arsenal', 'Set8_Aegis', 'Set8_UndergroundThe',
                                                 'Set8_StarGuardian', 'Set8_Prankster', 'Set8_Heart', 'Set8_Corrupted', 'Set8_Supers',
                                                 'Set8_Renegade', 'Set8_Channeler', 'Set8_Recon', 'Set8_Deadeye', 'Set8_Defender',
                                                'Set8_GenAE', 'Set8_Hacker']})
        self.champ_language = {self.champ.apiName[i] : self.champ.name[i] for i in range(self.champ.shape[0])}
        
        
    # 증강체 한글 변환
    
    def change_augment_name(self):
    
    
        change_results =[]

        for augments in self.game_result['augments']:

            change_result = []

            for augment in augments:

                change_augment = self.item_data.loc[self.item_data.apiName == augment, 'name'].values[0]
                change_result.append(change_augment)

            change_results.append(change_result)

        self.game_result.augments = change_results

        return self.game_result
    
    
    # trait dataframe 만들기
    
    
    def make_trait_dataframe(self,trait):
    
        trait = pd.DataFrame(trait)

        trait = trait[trait.tier_current!=0].drop(['style'], axis=1).reset_index(drop=True)

        return trait
    
    

    def change_language_trait(self,trait):

        name = []

        for i in trait['name']:
            change_name = self.trait_language.loc[self.trait_language['English']==i, 'Korean'].values[0]
            name.append(change_name)

        trait['name_kor'] = name
    
        return trait

    
    # Unit dataframe 만들기
    
    def make_unit_dataframe(self,unit):
    
        units = pd.DataFrame(unit)
        

        item_list = []
        champ_name = []
        for i in range(len(units['character_id'])):
            
            champ_name.append(self.champ_language[units['character_id'][i]])
            check = units['items'][i]

            check += [0]*(3-len(check))

            item_list.append(check)

        units['items'] = item_list

        item_name = []

        for item in item_list:

            check_item = []

            for i in item:
                if i==0:
                    check_item.append('None')
                else:
                    item_kor = self.item_data.loc[self.item_data.id==i, 'name'].values[0]
                    check_item.append(item_kor)
            item_name.append(check_item)


        units = pd.DataFrame(unit).drop(['itemNames', 'name'], axis=1)

        units['item_name'] = item_name
        units.character_id = list(map(lambda x: x.replace(self.version, ''), units.character_id))
        units['champ_name'] = champ_name

        return units
    
    
    # 수정된 데이터프레임 만들기
    
    
    def make_new_game_result(self):
    

        New_result = {'Game_id' : self.df['metadata']['match_id'],
                      'Player_id' : [],
                 'Placement' : [], 
                 'Time' : [],
                 'Last_round' : [],
                 "Damage_player" : [],
                 "Gold_left" : [],
                 "augments" : [],
                 "traits" : [],
                 "num_units" : [],
                 "tier_current" : [],
                 "units" : [],
                 "unit_tier" : [],
                 "item_name" : []}

        New_result['Player_id'] += (list(self.game_result.puuid))
        
        New_result['Placement'] +=(list(self.game_result.placement))

        New_result['Time']+=(list(self.game_result.time_eliminated))

        New_result['Last_round']+=(list(self.game_result.last_round))

        New_result['Damage_player']+=(list(self.game_result.total_damage_to_players))

        New_result['Gold_left']+=(list(self.game_result.gold_left))

        New_result['augments'] += list(self.game_result.augments)


        for player in range(self.game_result.shape[0]):

            trait_dataframe = pd.DataFrame(self.game_result.loc[player, 'traits'])
            trait_dataframe = self.make_trait_dataframe(trait_dataframe)
            trait_dataframe = self.change_language_trait(trait_dataframe)

            New_result['traits'].append(list(trait_dataframe.name_kor))
            New_result['num_units'].append(list(trait_dataframe.num_units))
            New_result['tier_current'].append(list(trait_dataframe.tier_current))


            unit_dataframe =  self.make_unit_dataframe(self.game_result.loc[player, 'units'])

            New_result['units'].append(list(unit_dataframe.champ_name))
            New_result['unit_tier'].append(list(unit_dataframe.tier))
            New_result['item_name'].append(list(unit_dataframe.item_name))


        New_result = pd.DataFrame(New_result)

        return pd.DataFrame(New_result)
    
    
    # 실제 method 입력시 trait dataframe을 제공해주는 함수
    
    def make_trait_result(self, number):
        
        trait1 = self.make_trait_dataframe(self.game_result.loc[number, 'traits'])
        trait1 = self.change_language_trait(trait1)
        trait1['Game_id'] = self.game_id
        trait1['Player_id'] = self.game_result.loc[number, 'puuid']
        trait1 = trait1[['Game_id', 'Player_id', 'name', 'num_units', 'tier_current', 'tier_total', 'name_kor']]
        
        return trait1
    
    # 실제 method 입력시 unit dataframe을 제공해주는 함수
    
    def make_unit_result(self, number):
        
        unit1 = self.make_unit_dataframe(self.game_result.loc[number, 'units'])
        unit1['Game_id'] = self.game_id
        unit1['Player_id'] = self.game_result.loc[number, 'puuid']
        unit1 = unit1[['Game_id', 'Player_id', 'character_id', 'champ_name', 'items', 'item_name']]
        return unit1

